In [ ]:
using Images, Colors, Distributions

function muestra(radio=2.0, centro=0.0im, alto=1000, ancho=1000)
    dx=(2.0*radio)/(1.0*ancho) #el ancho que representa un pixel
    dy=(2.0im*radio)/(1.0*alto)#el alto que representa un pixel
    z0 = centro + radio*(1.0im - 1.0) #el primer valor de la muestra es la esquina superior izquierda
    return [z0 + (x*dx) - (y*dy) for y in 1:alto, x in 1:ancho]
end

function moneda(N)
    ceil(Int64, round(rand(Uniform(-N-0.499, N+0.499))))
end

function paleta(mi_foto, paso, tamaño)
    aesthetics = load(mi_foto)
    alto, ancho =size(aesthetics)
    x0 = ceil(Int64, rand(Uniform(0,ancho)))
    y0 = ceil(Int64, rand(Uniform(0,alto)))
    paleta = zeros(RGB{N0f8}, tamaño)
    for i = 1:length(paleta)
        paleta[i] = aesthetics[mod(y0,alto)+1,mod(x0,ancho)+1]
        x0 = x0 + moneda(paso)
        y0 = y0 + moneda(paso)
    end
    return paleta
end

# Dinámica del seno

En este ejemplo vamos a considerar la función $$ f_{\lambda} (z) = \lambda \sin z. $$

Notese que el punto $0$ es fijo para esta función y su multiplicador es $f_{\lambda}' (0) = \lambda \cos 0 = \lambda$.

En el caso $|\lambda| < 1$ podemos dibujar la cuenca de atraccion del 0.

In [ ]:
function trampa_seno_atractor(Z, λ, paleta, eps=0.05, TOPE=1000)
    cont = 0
    while cont <= TOPE
        if abs(Z) < eps
            n = 1 + mod(cont, length(paleta))
            return paleta[n]
        elseif abs(imag(Z)) > 100.0
            return RGB{Float32}(1.0, 1.0, 1.0)
        end
        Z = λ*sin(Z)
        cont += 1
    end
    return RGB{Float32}(0.0, 0.0, 0.0) #color negro
end

function seno_atractor(λ, paleta, eps=0.05, TOPE=1000, radio=8.0, centro=0.0, alto=1000, ancho=1000)
    if abs(λ) >= 1.0
        println("Argumento no valido")
        return
    end
    broadcast(Z -> trampa_seno_atractor(Z, λ, paleta, eps, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
mis_colores = paleta("aesthetic.jpg", 8, 2000)

In [ ]:
seno_atractor(-0.73-0.66im, mis_colores, 0.05, 2000, 5.0)

Ahora consideremos el caso $\lambda = e^{2 \pi i \xi}$ con $\xi \in \mathbb{Q}$.

Aunque hay una cantidad infinita de puntos críticos solo hay dos valores críticos, y son $\pm \lambda$. En este caso usaremos una trampa similar a la que usamos para polinomios cuadraticos con punto fijo parabólico. Dado que $\sin$ es una función impar, las órbitas de $\pm \lambda$ son simétricas.

In [ ]:
function trampa_seno_parabolico(Z, λ, dir1, dir2, wdt, colores, eps=0.05, TOPE=5000)
    cont = 0
    while cont <= TOPE
        if abs(Z) < eps && (angle(Z) > dir1 - wdt && angle(Z) < dir1 + wdt)
            n = 1 + mod(cont, length(colores))
            return colores[n]
        elseif abs(Z) < eps && (angle(Z) > dir2 - wdt && angle(Z) < dir2 + wdt)
            n = length(colores) - mod(cont, length(colores))
            return colores[n]
        elseif abs(imag(Z)) > 100.0
            return RGB{Float32}(1.0, 1.0, 1.0) #color blanco
        end
        Z = λ*sin(Z)
        cont += 1
    end
    return RGB{Float32}(0.0, 0.0, 0.0) #color negro
end

function seno_parabolico(p, q, colores, eps=0.05, TOPE=5000, radio=8.0, centro=0.0, alto=1000, ancho=1000)
    λ = cos(2.0*pi*p/q) + sin(2.0*pi*p/q)*1.0im
    cr = λ
    for i = 1:TOPE
        if abs(cr) < eps
            break
        end
        cr = λ*sin(cr)
    end
    dir = angle(cr)
    dir2 = angle(-cr)
    wdt = pi*eps/(2.0*q)
    broadcast(Z -> trampa_seno_parabolico(Z, λ, dir, dir2, wdt, colores, eps, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
seno_parabolico(2, 3, mis_colores, 0.5)

De manera similar al caso cuadrático, suponiendo que conocemos el parámetro $\theta$ tal que $$ e^{2 \pi i \theta} \sin z$$ tiene un disco de Siegel centrado en $0$ podemos aplicar una trampa.

In [ ]:
function trampa_seno_siegel(Z, λ, rad, colores, TOPE=5000)
    cont = 0
    while cont <= TOPE
        if abs(Z) < rad
            n = 1 + mod(cont, length(colores))
            return colores[n]
        elseif abs(imag(Z)) > 100.0
            return RGB{Float32}(1.0, 1.0, 1.0) #blanco si diverge a infinito
        end
        Z = λ*sin(Z)
        cont += 1
    end
    return RGB{Float32}(0.0, 0.0, 0.0) #color negro si no concluye
end

function seno_siegel(θ, colores, TOPE=5000, radio=8.0, centro=0.0, alto=1000, ancho=1000)
    λ = cos(2.0*pi*θ) + sin(2.0*pi*θ)*1.0im
    cr = λ
    rad = 1000.0
    for i = 1:TOPE
        cr = λ*sin(cr)
        rad > abs(cr) ? rad = abs(cr) : nothing
    end
    broadcast(Z -> trampa_seno_siegel(Z, λ, rad, colores, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
seno_siegel(0.5*(sqrt(5.0) + 1.0), mis_colores)